In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score , precision_score , recall_score , f1_score
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [2]:
df = pd.read_csv("IMDB Dataset.csv")
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
df.isna().sum()

review       0
sentiment    0
dtype: int64

In [5]:
type(df['sentiment'][0])

str

In [6]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Lemmatization
def lemmatization(text):
    """Lemmatize the text."""
    if not isinstance(text, str):
        return ""
    lemmatizer = WordNetLemmatizer()
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

# Stop word removal
def remove_stop_words(text):
    """Remove stop words from the text."""
    if not isinstance(text, str):
        return ""
    stop_words = set(stopwords.words("english"))
    return " ".join([word for word in text.split() if word not in stop_words])

# Remove numbers
def removing_numbers(text):
    """Remove numbers from the text."""
    if not isinstance(text, str):
        return ""
    return re.sub(r'\d+', '', text)

# Lowercase
def lower_case(text):
    """Convert text to lower case."""
    if not isinstance(text, str):
        return ""
    return text.lower()

# Remove punctuations
def removing_punctuations(text):
    """Remove punctuations from the text."""
    if not isinstance(text, str):
        return ""
    return re.sub(f"[{re.escape(string.punctuation)}]", " ", text)

# Remove URLs
def removing_urls(text):
    """Remove URLs from the text."""
    if not isinstance(text, str):
        return ""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub("", text)

# Normalize text pipeline
def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].fillna("").astype(str)  # Handle None/NaN
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [7]:
df = normalize_text(df)
df.head()

,review,sentiment
0,one reviewer mentioned watching oz episode hoo...,positive
1,wonderful little production br br the filming ...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically there s family little boy jake think...,negative
4,petter mattei s love time money visually stunn...,positive


In [8]:
df['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [9]:
x = df['sentiment'].isin(['positive' , 'negative'])
df = df[x]

In [10]:
df['sentiment'] = df['sentiment'].map({'positive':1 , 'negative':0})
df.head()

,review,sentiment
0,one reviewer mentioned watching oz episode hoo...,1
1,wonderful little production br br the filming ...,1
2,thought wonderful way spend time hot summer we...,1
3,basically there s family little boy jake think...,0
4,petter mattei s love time money visually stunn...,1


In [11]:
vectorizer = CountVectorizer(max_features=50)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [12]:
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size=0.2 , random_state=42)

In [ ]:
import dagshub
mlflow.set_tracking_uri("https://dagshub.com/Pandharimaske/MLOPs_Project.mlflow")
dagshub.init(repo_owner='Pandharimaske', repo_name='MLOPs_Project', mlflow=True)

mlflow.set_experiment("Logistic Regression Baseline")

Accessing as Pandharimaske

Initialized MLflow to track repo "Pandharimaske/MLOPs_Project"

Repository Pandharimaske/MLOPs_Project initialized!

2025/09/15 15:58:21 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/c14917f9d9414acdb92624f6b5ea479e', creation_time=1757932101805, experiment_id='0', last_update_time=1757932101805, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [ ]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        # logging.info("Saving and logging the model...")
        # mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)

2025-09-15 16:05:06,600 - INFO - Starting MLflow run...
2025-09-15 16:05:06,859 - WARNING - Retrying (Retry(total=6, connect=7, read=6, redirect=7, status=7)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /Pandharimaske/MLOPs_Project.mlflow/api/2.0/mlflow/runs/create
2025-09-15 16:05:07,391 - INFO - Logging preprocessing parameters...
2025-09-15 16:05:08,809 - INFO - Initializing Logistic Regression model...
2025-09-15 16:05:08,811 - INFO - Fitting the model...
2025-09-15 16:05:08,993 - INFO - Model training complete.
2025-09-15 16:05:08,994 - INFO - Logging model parameters...
2025-09-15 16:05:09,425 - INFO - Making predictions...
2025-09-15 16:05:09,431 - INFO - Calculating evaluation metrics...
2025-09-15 16:05:09,451 - INFO - Logging evaluation metrics...
2025-09-15 16:05:14,754 - INFO - Saving and logging the model...
2025/09/15 16:05:14 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025-0

🏃 View run delightful-kit-94 at: https://dagshub.com/Pandharimaske/MLOPs_Project.mlflow/#/experiments/0/runs/52546fd9391b4b2db3ba7bb676335629
🧪 View experiment at: https://dagshub.com/Pandharimaske/MLOPs_Project.mlflow/#/experiments/0
